In [1]:
import findspark
findspark.init()

In [2]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
data = spark.read.csv("Absenteeism_at_work.csv", header = True, sep=",")

In [4]:
data = data.withColumn("MOA", data["Month of absence"] - 0). \
            withColumn("label", data['Height'] - 0). \
            withColumn("ROA", data["Reason for absence"] - 0). \
            withColumn("distance", data["Distance from Residence to Work"] - 0). \
            withColumn("BMI", data["Body mass index"] - 0)

In [5]:
from pyspark.ml.feature import VectorAssembler
assem = VectorAssembler(inputCols=["label", "MOA"], outputCol='features')

In [6]:
data = assem.transform(data)

In [16]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

In [17]:
from pyspark.ml.feature import VectorIndexer
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

In [18]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [9]:
from pyspark.ml.classification import NaiveBayes
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

In [19]:
from pyspark.ml.feature import IndexToString
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [20]:
from pyspark.ml import Pipeline
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, nb, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [10]:
# train the model
#model = nb.fit(train)

In [21]:
# select example rows to display.
predictions = model.transform(test)

In [22]:
# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

+--------------+-----+-----------+
|predictedLabel|label|   features|
+--------------+-----+-----------+
|         196.0|172.0|[172.0,7.0]|
|         168.0|172.0|[172.0,6.0]|
|         196.0|172.0|[172.0,8.0]|
|         168.0|172.0|[172.0,3.0]|
|         168.0|172.0|[172.0,3.0]|
+--------------+-----+-----------+
only showing top 5 rows



In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")

In [24]:
y_true = data.select("BMI").rdd.flatMap(lambda x: x).collect()
y_pred = data.select("ROA").rdd.flatMap(lambda x: x).collect()

In [29]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

nbModel = model.stages[2]

In [30]:
print(nbModel)  # summary only
print("Naive Bayes - Test Accuracy = %g" % (accuracy))
print("Naive Bayes - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Naive Bayes Model is :\n" + str(confusionmatrix))

print("The precision score for Naive Bayes Model is: " + str(precision))

print("The recall score for Naive Bayes Model is: " + str(recall))

NaiveBayes_553be785f960
Naive Bayes - Test Accuracy = 0
Naive Bayes - Test Error = 1
The Confusion Matrix for Naive Bayes Model is :
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
The precision score for Naive Bayes Model is: 0.02972972972972973
The recall score for Naive Bayes Model is: 0.02972972972972973
